In [ ]:
# ============================ 0) Imports ============================
import os, re, gc, glob, math, zlib, base64, numpy as np, pandas as pd, matplotlib.pyplot as plt
from urllib.parse import urlparse
from collections import Counter, defaultdict

from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 선택적 의존성: 더 정밀한 HTML 파싱용 (없으면 정규식 폴백)
try:
    from bs4 import BeautifulSoup
except Exception:
    BeautifulSoup = None

# eTLD+1 계산(있으면 tldextract, 없으면 간단 버전)
try:
    import tldextract
except Exception:
    tldextract = None

def etld1(host: str) -> str:
    host = (host or "").lower()
    if not host: return ""
    if tldextract:
        ext = tldextract.extract(host)
        return f"{ext.domain}.{ext.suffix}" if ext.suffix else ext.domain
    parts = host.split(".")
    return ".".join(parts[-2:]) if len(parts) >= 2 else host

# 이전 세션/모델 정리 (시드 고정 없음!)
try: del model
except: pass
gc.collect()
keras.backend.clear_session()


# ============================ 1) Config =============================
PHISHING_DIR = r"C:/Users/wja65/phishing/phishing"          # 피싱 폴더(HTML들이 쭉)
NORMAL_DIR   = r"C:/Users/wja65/phishing/normal/normal"      # 정상 폴더(HTML들이 쭉)

# ★ 개수 제한 (None 이면 전부 사용)
NORMAL_LIMIT   = None   # 예: 6000, None
PHISHING_LIMIT = None   # 예: 1500, None

# 학습 하이퍼파라미터
BATCH_SIZE, EPOCHS = 256, 60
UNITS = [256, 128, 128, 64, 64, 32]
DROPOUT, NOISE_STD, LR, WD, CLIPNORM = 0.30, 0.01, 3e-4, 1e-4, 1.0

# 저장 경로
SAVE_DIR = r"C:/Users/wja65/phishing/outputs_html"
os.makedirs(SAVE_DIR, exist_ok=True)
SAVE_LAST = True
LAST_SAVE_PATH = os.path.join(SAVE_DIR, "last.csv")
PLOT_LOSS_PATH = os.path.join(SAVE_DIR, "plot_loss.png")
PLOT_ACC_PATH  = os.path.join(SAVE_DIR, "plot_acc.png")
PLOT_AUC_PATH  = os.path.join(SAVE_DIR, "plot_auc.png")
REPORT_PATH    = os.path.join(SAVE_DIR, "classification_report.txt")
CONFMAT_PATH   = os.path.join(SAVE_DIR, "confusion_matrix.csv")


# ======================= 2) HTML 파일 탐지(확장자 無 대응) ==================
HTML_EXTS = {".html", ".htm", ".xhtml", ".mhtml", ".shtml"}
HTML_HEAD_RE = re.compile(r"<!doctype\s+html|<html\b|<head\b|<meta\b|<title\b|<body\b", re.IGNORECASE)

def is_probably_html(path: str, sniff=8192) -> bool:
    name = os.path.basename(path).lower()
    _, ext = os.path.splitext(name)
    if ext in HTML_EXTS:
        return True
    if name.endswith("_html"):  # 확장자 없어도 `_html`로 끝나면 HTML로 간주
        return True
    try:
        with open(path, "rb") as f:
            chunk = f.read(sniff)
        txt = chunk.decode("utf-8", errors="ignore")
        if HTML_HEAD_RE.search(txt):
            return True
    except Exception:
        pass
    return False

def list_html_files(root_dir: str) -> list[str]:
    root_dir = os.path.abspath(root_dir)
    out = []
    for r, _, files in os.walk(root_dir):
        for fn in files:
            fp = os.path.join(r, fn)
            try:
                if os.path.getsize(fp) <= 0:
                    continue
            except Exception:
                continue
            if is_probably_html(fp):
                out.append(fp)
    return out

# 안전한 텍스트 로더 (혼합 인코딩 대응)
def read_text_any(path: str) -> str | None:
    for enc in ("utf-8", "utf-8-sig", "cp949", "euc-kr", "latin-1"):
        try:
            with open(path, "r", encoding=enc, errors="strict") as f:
                return f.read()
        except Exception:
            continue
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()
    except Exception:
        return None


# ====================== 3) URL 추정 & 피처 함수 ============================
_URL_RE = re.compile(r"https?://[^\s\"')<>]+", re.IGNORECASE)
_CANON_RE = re.compile(r'rel=["\']?canonical["\']?.*?href=["\']([^"\']+)["\']', re.IGNORECASE|re.DOTALL)
_OGURL_RE = re.compile(r'property=["\']og:url["\'].*?content=["\']([^"\']+)["\']', re.IGNORECASE|re.DOTALL)
_BASE_RE = re.compile(r'<base[^>]*href=["\']([^"\']+)["\']', re.IGNORECASE)
_META_REFRESH_URL_RE = re.compile(r'http-equiv=["\']refresh["\'].*?url=([^"\';>]+)', re.IGNORECASE)

def guess_url_from_html(text: str, fallback_path: str) -> str:
    html = text or ""
    for regex in (_CANON_RE, _OGURL_RE, _BASE_RE, _META_REFRESH_URL_RE):
        m = regex.search(html)
        if m:
            return m.group(1).strip()
    m = _URL_RE.search(html)
    if m:
        return m.group(0).strip()
    return "file://" + fallback_path.replace("\\", "/")

def shannon_entropy(s: str) -> float:
    if not s: return 0.0
    counts = Counter(s)
    n = len(s)
    return -sum((c/n) * math.log2(c/n) for c in counts.values())

JS_REDIRECT_SIGNS = [
    "location.href", "location.replace", "window.location", "top.location",
    "document.location", "self.location", "location.assign", "meta http-equiv=\"refresh\""
]
OBFUSCATION_SIGNS = [
    "atob(", "btoa(", "fromCharCode", "unescape(", "eval(", "Function(", "setTimeout(",
    "String.fromCharCode", "document.write(", "obfuscate", "charCodeAt"
]
SUS_KEYWORDS = [
    "login","sign in","signin","verify","password","passcode","otp",
    "bank","account","billing","security","update","restricted","unlock",
    "coin","wallet","metamask","seed","mnemonic"
]

def extract_urls_bs4(soup):
    urls = []
    for tag in soup.find_all(["a","link","img","script","iframe","source","video","audio","form"]):
        for attr in ["href","src","action","data-src"]:
            u = tag.get(attr)
            if not u: continue
            if isinstance(u, list): 
                for x in u:
                    if isinstance(x, str) and x.startswith(("http://","https://")):
                        urls.append(x)
            elif isinstance(u, str) and u.startswith(("http://","https://")):
                urls.append(u)
    return urls

def extract_all_http_urls(text: str) -> list[str]:
    if BeautifulSoup:
        try:
            soup = BeautifulSoup(text, "html.parser")
            urls = extract_urls_bs4(soup)
            # 정규식 보조
            urls += _URL_RE.findall(text)
            return list({u for u in urls if u.startswith(("http://","https://"))})
        except Exception:
            pass
    return list({u for u in _URL_RE.findall(text) if u.startswith(("http://","https://"))})

def url_lex(url: str) -> dict:
    s = str(url or "")
    p = urlparse(s)
    host = p.hostname or ""
    digits = sum(ch.isdigit() for ch in s)
    return {
        "lex_len_url": len(s),
        "lex_len_host": len(host),
        "lex_nb_dot": s.count("."),
        "lex_nb_dash": s.count("-"),
        "lex_nb_slash": s.count("/"),
        "lex_nb_qm": s.count("?"),
        "lex_nb_eq": s.count("="),
        "lex_ratio_digits": digits / max(1, len(s)),
        "lex_nb_subdomains": max(0, host.count(".") - 1),
        "lex_has_ip_host": int(bool(re.match(r"^\d{1,3}(\.\d{1,3}){3}$", host))),
        "lex_path_len": len(p.path or ""),
        "lex_query_len": len(p.query or ""),
        "lex_port_nonstd": int(p.port not in [80, 443, None]),
    }

def extract_html_features(text: str) -> dict:
    if not text: text = ""
    low = text.lower()
    feats = {
        # 길이/비율/엔트로피
        "html_len": len(text),
        "html_nonascii_ratio": sum(ord(c)>127 for c in text)/max(1,len(text)),
        "html_percent_encoded_ratio": low.count("%")/max(1,len(text)),
        "html_entropy": shannon_entropy(text[:20000]),  # 너무 길면 클립

        # 태그/폼/입력
        "html_nb_script": 0, "html_nb_iframe": 0, "html_nb_form": 0,
        "html_nb_input": 0, "html_nb_pw_input": 0, "html_nb_link": 0,
        "html_nb_button": 0,

        # 이벤트 핸들러/난독화/데이터 URI
        "html_has_onclick": int("onclick=" in low),
        "html_has_data_uri": int("data:image" in low),
        "html_has_eval": int("eval(" in low),
        "html_nb_obfuscation_kw": sum(1 for sig in OBFUSCATION_SIGNS if sig.lower() in low),

        # 의심 키워드
        "html_nb_sus_kw": sum(low.count(kw) for kw in SUS_KEYWORDS),

        # JS 리다이렉션 시그널
        "html_nb_js_redirect": sum(low.count(sig) for sig in JS_REDIRECT_SIGNS),
    }

    if BeautifulSoup:
        try:
            soup = BeautifulSoup(text, "html.parser")
            feats["html_nb_script"]  = len(soup.find_all("script"))
            feats["html_nb_iframe"]  = len(soup.find_all("iframe"))
            feats["html_nb_form"]    = len(soup.find_all("form"))
            inputs = soup.find_all("input")
            feats["html_nb_input"]   = len(inputs)
            feats["html_nb_pw_input"]= sum(1 for i in inputs if (i.get("type") or "").lower()=="password")
            feats["html_nb_link"]    = len(soup.find_all("a"))
            feats["html_nb_button"]  = len(soup.find_all("button"))
        except Exception:
            pass
    else:
        feats["html_nb_script"]   = len(re.findall(r"<script\b", text, re.IGNORECASE))
        feats["html_nb_iframe"]   = len(re.findall(r"<iframe\b", text, re.IGNORECASE))
        feats["html_nb_form"]     = len(re.findall(r"<form\b", text, re.IGNORECASE))
        feats["html_nb_input"]    = len(re.findall(r"<input\b", text, re.IGNORECASE))
        feats["html_nb_pw_input"] = len(re.findall(r'<input[^>]*type=["\']password', text, re.IGNORECASE))
        feats["html_nb_link"]     = len(re.findall(r"<a\b", text, re.IGNORECASE))
        feats["html_nb_button"]   = len(re.findall(r"<button\b", text, re.IGNORECASE))
    return feats

def external_link_features(main_url: str, urls: list[str]) -> dict:
    # 메인 도메인 추정
    p = urlparse(main_url)
    main_host = p.hostname or ""
    main_e = etld1(main_host) if main_host else ""

    ehosts = [etld1(urlparse(u).hostname or "") for u in urls if (u and urlparse(u).scheme in ("http","https"))]
    ehosts = [e for e in ehosts if e]  # 빈값 제거
    unique_ext = {e for e in ehosts if e != main_e and e != ""}
    return {
        "link_total": len(urls),
        "link_unique_domain": len(set(ehosts)),
        "link_external_domain": len(unique_ext),
        "link_external_ratio": (len(unique_ext)/max(1,len(set(ehosts)))),
    }

def redirection_heuristics(text: str) -> dict:
    low = (text or "").lower()
    patterns = [
        r'\bwindow\.location\s*=\s*',
        r'\blocation\.href\s*=\s*',
        r'\bmeta\s+http-equiv=["\']refresh',
        r'\bsettimeout\s*\(',
        r'\btop\.location\s*=\s*',
    ]
    return {"redir_hits": sum(len(re.findall(p, low)) for p in patterns)}


# ======================= 4) 폴더 로드 & 라벨링 ============================
def load_html_dir(dir_path: str, label: int, label_name: str, limit: int|None) -> pd.DataFrame:
    files = list_html_files(dir_path)
    if not files:
        print(f"[warn] HTML-like 파일 없음: {dir_path}")
        return pd.DataFrame(columns=["filepath","url","label","label_name","html"])
    idx = np.arange(len(files))
    np.random.shuffle(idx)         # 시드 없음
    if limit is not None:
        idx = idx[:min(limit, len(idx))]
    chosen = [files[i] for i in idx]

    rows = []
    for fp in chosen:
        text = read_text_any(fp)
        if not text:
            continue
        url = guess_url_from_html(text, fallback_path=fp)
        rows.append({"filepath": fp, "url": url, "label": label, "label_name": label_name, "html": text})
    return pd.DataFrame(rows)

df_normal  = load_html_dir(NORMAL_DIR,   label=0, label_name="normal",   limit=NORMAL_LIMIT)
df_phish   = load_html_dir(PHISHING_DIR, label=1, label_name="phishing", limit=PHISHING_LIMIT)

last = pd.concat([df_normal, df_phish], ignore_index=True).sample(frac=1.0).reset_index(drop=True)

print("Counts after limiting:")
print("  normal:", len(df_normal), "| phishing:", len(df_phish))
print("last shape:", last.shape)
print("class value_counts:\n", last["label"].value_counts())

if last["label"].nunique() < 2:
    raise SystemExit("❌ 두 클래스(0/1)가 모두 필요합니다. 폴더/제한 설정 확인.")

if SAVE_LAST:
    last[["filepath","url","label","label_name"]].to_csv(LAST_SAVE_PATH, index=False, encoding="utf-8")
    print("✅ last saved to:", LAST_SAVE_PATH)


# ============================ 5) Feature Set =========================
# 5-1) URL lexical (대표 URL 기준)
lex = last["url"].map(url_lex).apply(pd.Series)

# 5-2) HTML 구조 & 난독화/리다이렉션 지표
html_feats = last["html"].map(extract_html_features).apply(pd.Series)

# 5-3) 문서 내 모든 외부 링크 분석
all_urls = last["html"].map(extract_all_http_urls)
ext_link_feats = []
for main_u, urls in zip(last["url"], all_urls):
    f = external_link_features(main_u, urls)
    ext_link_feats.append(f)
ext_link_feats = pd.DataFrame(ext_link_feats)

# 5-4) 추가 리다이렉션 시그널
redir_feats = last["html"].map(redirection_heuristics).apply(pd.Series)

# 합치기
last_feats = pd.concat(
    [last[["url","label","label_name","filepath"]],
     lex, html_feats, ext_link_feats, redir_feats],
    axis=1
)

feature_cols = [c for c in last_feats.columns
                if c.startswith(("lex_","html_","link_","redir_"))]
X_all = last_feats[feature_cols].apply(pd.to_numeric, errors="coerce").fillna(0).astype("float32").to_numpy()
y_all = last_feats["label"].astype("float32").to_numpy()

print("n_features:", len(feature_cols))


# ====== 6) Split (도메인/폴더 기반 그룹으로 누수 방지: 동일그룹은 한쪽에만) ======
def group_from(u: str, fp: str) -> str:
    host = urlparse(str(u)).hostname
    if host:
        return etld1(host)
    # file:// 케이스 → 상위 폴더명으로 그룹
    return os.path.basename(os.path.dirname(fp)) or "nogroup"

groups = pd.Series([group_from(u, fp) for u, fp in zip(last_feats["url"], last_feats["filepath"])],
                   index=last_feats.index)

gss = GroupShuffleSplit(n_splits=1, train_size=0.8)  # 시드 지정 안 함
tr_i, va_i = next(gss.split(X_all, y_all, groups=groups))

X_train, y_train = X_all[tr_i], y_all[tr_i]
X_val,   y_val   = X_all[va_i], y_all[va_i]
print("Split ->", X_train.shape, X_val.shape)

# 클래스 가중치
classes = np.unique(y_train.astype(int))
cw = compute_class_weight(class_weight="balanced", classes=classes, y=y_train.astype(int))
class_weight = {int(k): float(v) for k, v in zip(classes, cw)}
print("class_weight:", class_weight)


# ============================ 7) Model =====================================
def build_res_mlp(input_dim: int, X_train_sample: np.ndarray) -> keras.Model:
    inp  = keras.Input(shape=(input_dim,), name="features")
    norm = layers.Normalization(name="norm")
    norm.adapt(X_train_sample)  # train 통계만 사용

    x = norm(inp)
    x = layers.GaussianNoise(NOISE_STD)(x)
    x = layers.Dropout(DROPOUT)(x)

    def block(x, units):
        h = layers.Dense(units, activation="relu",
                         kernel_regularizer=keras.regularizers.L2(WD))(x)
        h = layers.Dropout(DROPOUT)(h)
        h = layers.Dense(units, activation="relu",
                         kernel_regularizer=keras.regularizers.L2(WD))(h)
        if x.shape[-1] != units:
            x = layers.Dense(units, activation=None)(x)
        return layers.Add()([x, h])

    x = layers.Dense(UNITS[0], activation="relu")(x)
    for u in UNITS[1:]:
        x = block(x, u)

    out = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inp, out, name="phish_resmlp")

    opt  = keras.optimizers.Adam(learning_rate=LR, clipnorm=CLIPNORM)
    model.compile(
        optimizer=opt,
        loss="binary_crossentropy",
        metrics=[keras.metrics.BinaryAccuracy(name="accuracy"),
                 keras.metrics.AUC(name="auc")]
    )
    return model

model = build_res_mlp(X_train.shape[1], X_train)
model.summary()

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=6,
                                  restore_best_weights=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor="val_auc", mode="max",
                                      factor=0.5, patience=3, verbose=1),
]


# ============================ 8) Train =====================================
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS, batch_size=BATCH_SIZE,
    class_weight=class_weight,
    callbacks=callbacks, verbose=1
)


# ============================ 9) Plot & Eval ===============================
def save_hist_plot(hist, keys, title, out_path, ylim=None, ylabel=None):
    plt.figure()
    plotted = False
    for k in keys:
        if k in hist.history:
            plt.plot(hist.history[k], label=k)
            plotted = True
    if not plotted:
        print(f"[warn] {keys} 없음, 플롯 생략")
        return
    plt.title(title); plt.xlabel("Epochs")
    plt.ylabel(ylabel if ylabel else keys[0])
    if ylim: plt.ylim(*ylim)
    plt.legend(); plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()
    print(f"📈 Saved plot -> {out_path}")

save_hist_plot(history, ["loss","val_loss"], "Loss", PLOT_LOSS_PATH, ylabel="Loss")
save_hist_plot(history, ["accuracy","val_accuracy"], "Accuracy", PLOT_ACC_PATH, ylim=(0,1), ylabel="Accuracy")
save_hist_plot(history, ["auc","val_auc"], "ROC-AUC", PLOT_AUC_PATH, ylim=(0,1), ylabel="AUC")

# Validation 성능 수치
val_prob = model.predict(X_val, verbose=0).ravel()
val_pred = (val_prob >= 0.5).astype(int)

target_names = ["normal(0)", "phishing(1)"]
report = classification_report(y_val.astype(int), val_pred, target_names=target_names, digits=4)
cm = confusion_matrix(y_val.astype(int), val_pred, labels=[0,1])
try:
    auc_val = roc_auc_score(y_val, val_prob)
except Exception:
    auc_val = float("nan")

print("\n=== Validation Report ===")
print(report)
print("Confusion Matrix (rows=true, cols=pred):\n", cm)
print(f"Val ROC-AUC: {auc_val:.4f}")

# 저장
with open(REPORT_PATH, "w", encoding="utf-8") as f:
    f.write("=== Validation Report ===\n")
    f.write(report + "\n\n")
    f.write("Confusion Matrix (rows=true, cols=pred):\n")
    f.write(pd.DataFrame(cm, index=["true_normal","true_phishing"], columns=["pred_normal","pred_phishing"]).to_string() + "\n\n")
    f.write(f"Val ROC-AUC: {auc_val:.6f}\n")
print(f"📝 Saved report -> {REPORT_PATH}")

pd.DataFrame(cm, index=["true_normal","true_phishing"], columns=["pred_normal","pred_phishing"])\
  .to_csv(CONFMAT_PATH, encoding="utf-8", index=True)
print(f"🧾 Saved confusion matrix -> {CONFMAT_PATH}")
